#  Week #7  -  Python Practice

### Indicating and Describing “Heavy” XYZ Customers

In [1]:
import os

import cPickle as pickle

import pandas as pd  # panda's nickname is pd

import numpy as np  # numpy as np

import sqlalchemy

from pandas import DataFrame, Series, Categorical     # for convenience
from sqlalchemy import create_engine
from sqlalchemy import schema
from sqlalchemy import inspect 	# helps us look at the tables in xyz.db


In [2]:
engine=create_engine('sqlite:///xyz.db')	    # the db is in my current working directory

In [3]:
insp=inspect(engine)		# get data about xyz that can be examined.

In [4]:
insp.get_table_names()

[u'xyzcust', u'xyztrans']

In [5]:
xyzcust=pd.read_sql_table('xyzcust',engine)
xyzcust[['YTD_SALES_2009','YTD_TRANSACTIONS_2009']].describe()


,YTD_SALES_2009,YTD_TRANSACTIONS_2009
count,30179.000000,30179.000000
mean,236.283972,0.844760
std,2117.042293,1.475401
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,207.000000,1.000000
max,351000.000000,36.000000


In [6]:
pct=[x/10. for x in range(1,10)]		# a simple list comprehension

In [7]:
xyzcust[['YTD_SALES_2009','YTD_TRANSACTIONS_2009']].describe(percentiles=pct)


,YTD_SALES_2009,YTD_TRANSACTIONS_2009
count,30179.000000,30179.000000
mean,236.283972,0.844760
std,2117.042293,1.475401
min,0.000000,0.000000
10%,0.000000,0.000000
20%,0.000000,0.000000
30%,0.000000,0.000000
40%,0.000000,0.000000
50%,0.000000,0.000000
60%,57.000000,1.000000


In [8]:
# xyzcust.BUYER_STATUS.<tab>	# pick the method you need.

xyzcust.BUYER_STATUS.describe  	# pick the method you need.

<bound method Series.describe of 0        INACTIVE
1          ACTIVE
2          ACTIVE
3        INACTIVE
4          ACTIVE
5          ACTIVE
6          ACTIVE
7          ACTIVE
8        INACTIVE
9        INACTIVE
10         ACTIVE
11         LAPSED
12         ACTIVE
13       INACTIVE
14         ACTIVE
15       INACTIVE
16         LAPSED
17         ACTIVE
18         ACTIVE
19         ACTIVE
20       INACTIVE
21         ACTIVE
22       INACTIVE
23       INACTIVE
24         ACTIVE
25         ACTIVE
26         LAPSED
27         ACTIVE
28         ACTIVE
29         LAPSED
           ...   
30149      ACTIVE
30150      ACTIVE
30151      ACTIVE
30152      ACTIVE
30153      ACTIVE
30154    INACTIVE
30155      LAPSED
30156      LAPSED
30157      ACTIVE
30158      LAPSED
30159    INACTIVE
30160      LAPSED
30161      LAPSED
30162      ACTIVE
30163      ACTIVE
30164    INACTIVE
30165      ACTIVE
30166      ACTIVE
30167      ACTIVE
30168    INACTIVE
30169      ACTIVE
30170      ACTIVE
30171    INAC

In [9]:
#xyzcust.BUYER_STATUS.count

buyer_status = Categorical(xyzcust['BUYER_STATUS'])


In [10]:
buyer_status.describe()

,counts,freqs
categories,,
ACTIVE,13332,0.441764
INACTIVE,8996,0.298088
LAPSED,7851,0.260148


In [11]:
buyer_status.unique

<bound method Categorical.unique of [INACTIVE, ACTIVE, ACTIVE, INACTIVE, ACTIVE, ..., ACTIVE, ACTIVE, INACTIVE, INACTIVE, ACTIVE]
Length: 30179
Categories (3, object): [ACTIVE, INACTIVE, LAPSED]>

In [12]:
buyer_status_ordered=buyer_status.reorder_categories(['LAPSED','INACTIVE','ACTIVE'])


In [13]:
buyer_status_ordered.describe()

,counts,freqs
categories,,
LAPSED,7851,0.260148
INACTIVE,8996,0.298088
ACTIVE,13332,0.441764


In [14]:
xyzcust.YTD_SALES_2009[buyer_status_ordered=="ACTIVE"].describe(percentiles=pct)


count     13332.000000
mean        534.864536
std        3160.079031
min           3.000000
10%          60.000000
20%          93.000000
30%         132.000000
40%         180.000000
50%         246.000000
60%         333.000000
70%         468.000000
80%         684.000000
90%        1154.700000
max      351000.000000
Name: YTD_SALES_2009, dtype: float64

In [15]:
xyzcust.YTD_SALES_2009[buyer_status_ordered=="ACTIVE"].hist(bins=30,range=(0,900))


In [16]:
xyzcust.YTD_SALES_2009[(xyzcust.BUYER_STATUS=="ACTIVE") & (xyzcust.YTD_SALES_2009<=0)].sum()


0

In [17]:
xyzcust.YTD_SALES_2009[(buyer_status_ordered=="ACTIVE") ].quantile(0.67)

423.0

In [18]:
heavyCut= 423  #heavyCut is a constant

In [19]:
heavyCat=Categorical(np.where(xyzcust.YTD_SALES_2009>heavyCut,1,0))

In [20]:
heavyCat.describe()

,counts,freqs
categories,,
0,25795,0.854733
1,4384,0.145267


In [21]:
heavyCat.rename_categories(['regular','heavy'],inplace=True)

In [22]:
heavyCat.describe()

,counts,freqs
categories,,
regular,25795,0.854733
heavy,4384,0.145267


In [23]:
buyerType=pd.get_dummies(heavyCat)

In [24]:
buyerType.head()

,regular,heavy
0,1,0
1,0,1
2,1,0
3,1,0
4,1,0


In [25]:
xyzcust[(xyzcust.YTD_SALES_2009==max(xyzcust.YTD_SALES_2009))|(xyzcust.YTD_TRANSACTIONS_2009==max(xyzcust.YTD_TRANSACTIONS_2009))]


,index,ACCTNO,ZIP,ZIP4,LTD_SALES,LTD_TRANSACTIONS,YTD_SALES_2009,YTD_TRANSACTIONS_2009,CHANNEL_ACQUISITION,BUYER_STATUS,ZIP9_SUPERCODE
1756,1773,PPHDLSPGP,60067,8002,33993.0,175,6429.0,36,CB,ACTIVE,600678002
1855,1872,LLSYGYDAY,60074,0,2247750.0,84,351000.0,19,CB,ACTIVE,600746495


In [26]:
pct5, pct95 = xyzcust.YTD_SALES_2009[(buyer_status_ordered=="ACTIVE") ].quantile([0.05,0.95])


In [27]:
pct5, pct95

(42.0, 1798.3499999999967)

In [28]:
winSales=xyzcust.YTD_SALES_2009		# just making a copy here.

In [29]:
winSales[winSales>pct95].head()

12      2064.0
24      1875.0
77      1815.0
137    14448.0
228     5586.0
Name: YTD_SALES_2009, dtype: float64

In [30]:
len(winSales[winSales>pct95])

667

In [31]:
winSales[winSales>pct95]=pct95  		# collapsing the extreme high values

/Users/Zeeshan/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [32]:
#Sanity Test
winSales[winSales==pct95].head()

12     1798.35
24     1798.35
77     1798.35
137    1798.35
228    1798.35
Name: YTD_SALES_2009, dtype: float64

In [33]:
winSales[winSales<pct5].head()

0     0.0
3     0.0
7    33.0
8     0.0
9     0.0
Name: YTD_SALES_2009, dtype: float64

In [34]:
winSales[winSales<pct5]=pct5 # smooshing up the low values

/Users/Zeeshan/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [35]:
winSales[winSales==pct5].head()

0    42.0
3    42.0
7    42.0
8    42.0
9    42.0
Name: YTD_SALES_2009, dtype: float64

In [36]:
xyzcust['heavyCat']=heavyCat

In [37]:
# Sanity Test
xyzcust[['ACCTNO', 'YTD_SALES_2009', 'YTD_SALES_2009', 'heavyCat']].head()


,ACCTNO,YTD_SALES_2009,YTD_SALES_2009,heavyCat
0,WDQQLLDQL,42.0,42.0,regular
1,WQWAYHYLA,1263.0,1263.0,heavy
2,GSHAPLHAW,129.0,129.0,regular
3,PGGYDYWAD,42.0,42.0,regular
4,LWPSGPLLS,72.0,72.0,regular


In [38]:
# In case you get an error (it appears it is a bug in pandas.crosstab latest package release) when you execute the following command 
#
# pd.crosstab(xyzcust.heavyCat,xyzcust.CHANNEL_ACQUISITION,margins=True)
#
# Try the following command instead

pd.crosstab(xyzcust.heavyCat.astype('string'),xyzcust.CHANNEL_ACQUISITION.astype('string'),rownames=['heavyCat'], colnames=['CHANNEL_ACQUISITION'], margins=True)


CHANNEL_ACQUISITION,CB,IB,RT,All
heavyCat,,,,
heavy,356,703,3325,4384
regular,1506,3472,20817,25795
All,1862,4175,24142,30179


# Deliverable : If you were to define the heavies as those who have spent more than at least 51% of all ACTIVEs in 2009, what will be the heavyCut value?

In [39]:
xyzcust.YTD_SALES_2009[(buyer_status_ordered=="ACTIVE") ].quantile(0.51)

252.0